# Experiment Set Up

## Context and Purpose of the Experiment

This worksheet is designed to validate the behavior of privileges in Snowflake, specifically addressing whether granting privileges on a schema automatically propagates to the objects (e.g., tables and views) contained within it. The experiment stems from the need to confirm the following hypothesis:

> "Granting privileges on a schema does not inherently grant the same privileges on the objects contained within that schema."

---

## Experiment Setup

- **Database and Schema**: A database named `playground` and a schema named `PUBLIC` were created to serve as the container for the experiment.
- **Table Creation**: A table named `ORDERS` was added to the `PUBLIC` schema to test access to objects within the schema.
- **Role Setup**: A role, `ANALYST_ROLE`, was created and assigned to a user. This role was granted `USAGE` privileges on both the database and schema.
- **Access Test**: After granting `USAGE` privileges, the ability of the role to query the table without explicitly granting privileges on the table was tested.
- **Explicit Privilege Grant**: To confirm the hypothesis, the `SELECT` privilege was explicitly granted on the `ORDERS` table, and the role's access was retested.

---

## Goals

1. Validate whether `USAGE` privileges on a schema extend to objects within the schema.
2. Demonstrate the necessity of explicit privilege grants for accessing schema-contained objects.
3. Clarify privilege hierarchy and ensure alignment with Snowflake's access control model.

---

## Conducting the Experiment

### **Setup Phase**
- Created the database, schema, table, and role.
- Granted the role `USAGE` privileges on the database and schema.

### **Testing Phase**
- Verified whether the role could query the table after receiving `USAGE` privileges.

### **Validation Phase**
- Explicitly granted the `SELECT` privilege on the table.
- Retested the role's ability to query the table.

---

## Results

Granting Schema Access Does grant Access to Tables

## Run Experiment

In [ ]:
USE ROLE ACCOUNTADMIN;
SET my_user = (SELECT CURRENT_USER());
SELECT $my_user;

DROP TABLE IF EXISTS playground.PUBLIC.ORDERS;
CREATE TABLE playground.PUBLIC.ORDERS (
    ORDER_ID INT,
    ORDER_DATE DATE,
    CUSTOMER_ID INT,
    AMOUNT DECIMAL(10, 2)
);

DROP ROLE IF EXISTS ANALYST_ROLE;
CREATE ROLE ANALYST_ROLE;

GRANT USAGE ON DATABASE playground TO ROLE ANALYST_ROLE;
GRANT USAGE ON WAREHOUSE FN_DATA_ANALYTICS TO ROLE ANALYST_ROLE; -- grant this role a bit of computing resources
GRANT USAGE ON SCHEMA playground.PUBLIC TO ROLE ANALYST_ROLE;

GRANT ROLE ANALYST_ROLE TO USER IDENTIFIER($my_user);
SHOW GRANTS TO ROLE ANALYST_ROLE;

In [ ]:
SELECT CURRENT_ROLE();

Now we verify if we have modifying permisssion by switching to `ANALYST_ROLE` execute the following:


In [ ]:
USE ROLE ANALYST_ROLE;
SHOW GRANTS TO ROLE ANALYST_ROLE;

In [ ]:
INSERT INTO playground.PUBLIC.ORDERS (ORDER_ID, ORDER_DATE, CUSTOMER_ID, AMOUNT)
VALUES (1001, '2024-01-01', 456, 99.99);

Above code should show that we don't have privilege to select or modify the entry. Can we revoke the access so that ANALYST_ROLE is only given READ access to the public table?

In [ ]:
REVOKE USAGE ON SCHEMA playground.PUBLIC FROM ROLE ANALYST_ROLE;
SHOW GRANTS TO ROLE ANALYST_ROLE;

Above we tried to revoke ANALYST_ROLE privilege as a ANALYST but the privileges remain unchanged!

This is because **Snowflake does not allow a role to revoke its own permissions.**

In [ ]:
USE ROLE ACCOUNTADMIN;
REVOKE USAGE ON SCHEMA playground.PUBLIC FROM ROLE ANALYST_ROLE;
GRANT SELECT ON playground.PUBLIC.ORDERS TO ROLE ANALYST_ROLE;
SHOW GRANTS TO ROLE ANALYST_ROLE;

Now let's try modifying the table again by switching to ANALYST_ROLE and select some entries

In [ ]:
use role ANALYST_ROLE;
select * from playground.public.orders

Q: If I grant a role to select from the table but not the usage on the schema, then would he be able to select the entries?

A: No, the role will NOT be able to select from the table if it has SELECT privileges on the table but lacks USAGE on the schema.

**In Snowflake's privilege hierarchy, a role must have USAGE on the schema AND SELECT on the table to access the table's data.
Without USAGE on the schema, the role cannot even see the table, even if it has SELECT on the table.**

In [ ]:
USE ROLE ACCOUNTADMIN;
GRANT USAGE ON SCHEMA playground.PUBLIC TO ROLE ANALYST_ROLE;
USE ROLE ANALYST_ROLE;
select * from playground.public.orders

NOW finally ANALYST_ROLE can query the table! Let's clean the tables we created
## Cleaning

In [ ]:
USE ROLE ACCOUNTADMIN;
DROP TABLE IF EXISTS playground.PUBLIC.ORDERS;
DROP ROLE ANALYST_ROLE;